In [1]:
import json
from dataclasses import dataclass, asdict
from functools import partial
from typing import Callable

import gensim.models
import numpy as np
from nltk import word_tokenize
from scipy.spatial.distance import cosine
from sklearn.cluster import DBSCAN, OPTICS
from munch import DefaultMunch
from tqdm import tqdm

from src.clustering import embeddings
from src.clustering.embeddings.tagger import tag
from src.settings import MODELS_PATH, DATA_PATH
from src.utils import file_paths
from src.utils.merge_csv import merge

munch = DefaultMunch.fromDict

[INFO] Loading dictionaries from c:\users\uiqko\pyenvs\mint\lib\site-packages\pymorphy2_dicts_ru\data
[INFO] format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345
[INFO] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[INFO] NumExpr defaulting to 8 threads.


In [2]:
%load_ext autoreload
%autoreload 2

In [7]:
w2v: gensim.models.KeyedVectors = embeddings.get_source('web').load(MODELS_PATH / 'embeddings')

[INFO] loading projection weights from D:\projects\rpd-keyword-extraction\models\embeddings\web_upos_cbow_300_20_2017\web_upos_cbow_300_20_2017.bin.gz
[INFO] KeyedVectors lifecycle event {'msg': 'loaded (267540, 300) matrix of type float32 from D:\\projects\\rpd-keyword-extraction\\models\\embeddings\\web_upos_cbow_300_20_2017\\web_upos_cbow_300_20_2017.bin.gz', 'binary': True, 'encoding': 'utf8', 'datetime': '2022-03-29T13:05:57.158506', 'gensim': '4.1.2', 'python': '3.9.6 (tags/v3.9.6:db3ff76, Jun 28 2021, 15:26:21) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'load_word2vec_format'}


In [8]:
w2v.add_vector('<unk>', np.zeros(300))

c:\users\uiqko\pyenvs\mint\lib\site-packages\gensim\models\keyedvectors.py:478: UserWarning: Adding single vectors to a KeyedVectors which grows by one each time can be costly. Consider adding in batches or preallocating to the required size.
  warnings.warn(



267540

In [10]:
w2v.get_vector('<unk>')

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [28]:
df = merge(file_paths(DATA_PATH / 'raw' / 'data_validation'), columns_take=3)

In [29]:
len(df)

2344

In [30]:
df.head(10)

,filename,keywords,keywords_processed
0,(222331) 090303 Б1.54 Теория принятия решений....,"контроля исполнения,контроля,решений теории уп...","теории принятия решений,принятие решений теори..."
1,(222332) 090303 Б1.55 Проектная документация.p...,"программного обеспечения,документированием про...",правила редактирования научнотехнической докум...
2,(223535) 450304 Б1.41 Телекоммуникационные сис...,"vlan,ip,cisco,модуль интернет подключения межс...","vlan,ip,cisco,модуль интернет подключения межс..."
3,(223542) 450304 Б1.42 Организация процессов ра...,"современных систем управления базами данных,пр...","предметная область автоматизации,современные п..."
4,(223670) 450304 Б1.48 Анализ текстовой информа...,"основные принципы,подходы анализу текстовых да...","анализа текстовых данных,\nнеобходимые для реш..."
5,(223673) 450304 Б1.48 Глубокое обучение.pdf.ta...,"основ искусственного интеллекта,нейронные сети...","нейронные сети,введение,искусственного интелле..."
6,(223686) 450304 Б1.49 Язык Python для анализа ...,"series,matplotlib,csvjson,html,notebook,pandas...","matplotlib,\ncsvjson,\nhtml,dataframe,\nmatplo..."
7,(223735) 450304 Б1.51 Проектная документация.p...,документированием программного обеспечения тре...,оформления технических заданий декомпозировать...
8,(223738) 450304 Б1.52 Инженерия знаний_2.pdf.t...,"введение системы,методы средства проектировани...",методы средства проектирования программного об...
9,(223803) 450304 Б1.54 Креативные технологии в ...,"визуализации данных,данных средствами языков п...","визуализации данных средствами языков,языки пр..."


In [31]:
list(map(tokenize, df.iloc[0, 2].split(',')))

[[1235, 1363, 100],
 [1363, 100, 1235, 154],
 [2537, 3872],
 [324, 45163, 372, 698, 391],
 [542, 384]]

In [32]:
# keyword_indexes = df.head(10).keywords_processed.map(lambda kws: list(map(tokenize, kws.removesuffix(',').split(','))))

In [33]:
def group_embeddings(indexes, fn=None):
    fn = fn or partial(np.sum, axis=0)
    return fn([w2v.get_vector(index) for index in indexes])

In [69]:
@dataclass
class Params:
    v: float
    group_fn: str
    model: str
    model_params: dict

@dataclass
class Scores:
    compression: float = None
    mean_group_size: float = None

    def __init__(self, groups: list, text_by_embedding):
        self.compression = 1 - round(len(groups) / len(text_by_embedding), 3)
        self.mean_group_size = np.mean(list(map(len, groups)))

In [74]:
params = Params(
    v=0.7,
    group_fn='mean',
    model='DBSCAN',
    model_params=dict(
        eps=0.07,
        min_samples=2,
        metric='cosine'
    )
)

In [75]:
X = []
text_by_embedding = {}
group_fn = partial(getattr(np, params.group_fn), axis=0)

for kws in tqdm(df.dropna().keywords_processed):
    if isinstance(kws, str):
        for kw in kws.removesuffix(',').split(','):
            kw = kw.strip()

            if kw == '':
                break

            indexes = tokenize(kw)
            emb = group_embeddings(indexes, group_fn)
            key_emb = tuple(emb)
            # while key_emb in text_by_embedding:
            #     emb += 0.000001
            #     key_emb = tuple(emb)

            text_by_embedding[key_emb] = kw

            X.append(emb)

X = np.array(X)

100%|██████████| 2314/2314 [00:11<00:00, 202.38it/s]


In [76]:
model = DBSCAN(**params.model_params)
model.fit(X)

DBSCAN(eps=0.07, metric='cosine', min_samples=2)

In [77]:
groups = {}
other = []

for i, label in enumerate(model.labels_):
    if label == -1:
        other.append(text_by_embedding[tuple(X[i])])
    else:
        groups.setdefault(label, set()).add(text_by_embedding[tuple(X[i])])

In [78]:
scores = Scores(list(groups.values()) + [other], text_by_embedding)

print('Сжатие:', scores.compression)
print('Средний размер группы:', scores.mean_group_size)

Сжатие: 0.727
Средний размер группы: 4.3658947368421055


In [79]:
groups

{0: {'теории принятия управленческих решений', 'теория принятие решений'},
 1: {'анализ продукта'},
 2: {'внедрение программного обеспечения',
  'документационного сопровождения разработки программного обеспечения',
  'документирование \nразработки программного обеспечения',
  'методологии разработки программного обеспечения',
  'методы разработки программных программноаппаратных средств',
  'методы средства проектирования программного обеспечения',
  'программные средства для проектирования мэмс',
  'программные средства проектирования',
  'проектирование программного обеспечения',
  'разработка программного обеспечения',
  'разработки программного обеспечения документирование',
  'сопровождения разработки программного обеспечения',
  'средства проектирования программного обеспечения',
  'этапов разработки программного обеспечения документирование'},
 3: {'моделирования бизнеспроцессов методология ведения документооборота'},
 4: {'основные принципы дизайна телекоммуникационных сетей',

In [80]:
fname = f'clusters' \
            f'.v{params.v}' \
            f'.size_{len(df)}' \
            f'.{params.model}' \
            f'.json'

In [81]:
with open(DATA_PATH / 'results' / 'clusters' / fname, 'w', encoding='utf-8') as f:
    json.dump({
        'params': asdict(params),
        'scores': asdict(scores),
        'clusters': list(map(list, groups.values()))
    }, f, indent=4, ensure_ascii=False)